In [1]:
using Pipe

In [14]:
normInf(vec::Tuple{Int64, Int64}) =
  begin
    # return map((x->x*x), vec) |> sum |> sqrt
    # return map(abs, vec) |> sum
    return map(abs, vec) |> Base.splat(max)
  end

norm1(vec::Tuple{Int64, Int64}) =
  begin
    return map(abs, vec) |> sum
  end

normalize(vec::Tuple{Int64, Int64}) =
  begin
    size = normInf(vec)
    if (size == 0)
      return vec
    end
    return map(x -> div(x, size), vec)
  end

normalizeInf(vec::Tuple{Int64, Int64}) =
begin
  return map(x -> sign(x) * min(abs(x), 1), vec)
end


displayKnots(knots::Vector{Tuple{Int64, Int64}}) =
  begin
    nRows = 34
    nCols = 34
    # board = Matrix{Char}('_', nRows, nCols)
    board = fill('_', (nRows, nCols))    

    for i in length(knots):-1:1
      (x, y) = knots[i]
      board[div(nRows,2)-y, div(nCols,2)+x+1] = '0' + i - 1 
    end

    stringFormVec=Vector{String}()
    for i in 1:nRows
      strToCreate = ""
      for j in 1:nCols
        strToCreate *= board[i,j]
      end
      push!(stringFormVec, strToCreate)
    end

    stringForm = map(string, board)
    display(stringFormVec)
  end

displayKnots (generic function with 1 method)

In [3]:
offsetFromDirection = Dict('U' => (0, 1), 'D' => (0,-1), 'L' => (-1,0), 'R' => (1,0))

parseInputLine(line::String) =
  begin
    splitted = split(line)
    direction = splitted[1] |> first
    (dx, dy) = offsetFromDirection[direction]
    count = parse(Int64, splitted[2])
    return (count*dx, count*dy)
  end

parseInputLine (generic function with 1 method)

In [19]:
moves = Vector{Tuple{Int64, Int64}}()
for move in eachline("input.txt")
  @pipe move |> parseInputLine |> push!(moves, _)
end

In [7]:
# task 1
reachedPositions = Set{Tuple{Int64, Int64}}([(0,0)])
tPos = hPos = (0,0)

for move in moves
  # willMoveTowards = @pipe move |> map(sign, _) |> indexin(x-> x == 0, _) |> 3 - _
  (dx, dy) = normalize(move)
  for _ in 1:normInf(move)
    newHPos = hPos .+ (dx, dy)
    if normInf(tPos .- newHPos) > 1
      tPos = hPos
      push!(reachedPositions, tPos)
    end
    hPos = newHPos
  end
end

length(reachedPositions)

88

In [26]:
# task 2
reachedPositions = Set{Tuple{Int64, Int64}}([(0,0)])
knotsCount = 10
knotPositions = Vector{Tuple{Int64, Int64}}()
newKnotPositions = Vector{Tuple{Int64, Int64}}()
for _ in 1:knotsCount
  push!(knotPositions, (0,0))
  push!(newKnotPositions, (0,0))
end

for move in moves
  # display(move)
  # displayKnots(knotPositions)
  (dx, dy) = normalize(move)
  for _ in 1:normInf(move)
    if move == (-8, 0)
      displayKnots(knotPositions)
    end
    newKnotPositions[1] = knotPositions[1] .+ (dx, dy)
    
    for i in 1:knotsCount-1
      if normInf(knotPositions[i+1] .- newKnotPositions[i]) > 1
        newKnotPositions[i+1] = knotPositions[i]
        # println(i, "-", newKnotPositions[i+1])
        if norm1(knotPositions[i+1] .- newKnotPositions[i]) > 2
          newKnotPositions[i+1] = newKnotPositions[i+1] .+ (dx, dy)
          # println(i, "!!!-", newKnotPositions[i+1])
          # newKnotPositions[i+1] = knotPositions[i+1] .+ div.(newKnotPositions[i] .- knotPositions[i+1], 2)
        end
        if newKnotPositions[i+1] == newKnotPositions[i] || normInf(newKnotPositions[i+1] .- knotPositions[i+1]) > 1
          # newKnotPositions[i+1] = newKnotPositions[i+1] .- (dx, dy)
          # newKnotPositions[i+1] = div.(knotPositions[i] .+ newKnotPositions[i], 2)
          # newKnotPositions[i+1] = newKnotPositions[i+1] .+ (dy, dx)
          newKnotPositions[i+1] = knotPositions[i]
          # newKnotPositions[i+1] = knotPositions[i+1] .+ newKnotPositions[i] .+ (dx, dy) .- knotPositions[i]
        end
        # if norm1(newKnotPositions[i] .- knotPositions[i]) > 2
        #   if move == (-8, 0)
        #     println(i, "-", newKnotPositions[i+1])
        #   end    
        #   # newKnotPositions[i+1] = newKnotPositions[i+1] .+ (dx, dy)
        #   newKnotPositions[i+1] = newKnotPositions[i] .- (dy, dx)
        # end
      else newKnotPositions[i+1] = knotPositions[i+1]
      end
      if move == (-8, 0)
        println(i, "-", newKnotPositions[i+1])
      end
    end

    knotPositions = copy(newKnotPositions)
    push!(reachedPositions, knotPositions[knotsCount])
  end
end

reachedPositions

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________0_______"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________8765_________"
 "_________________4________"
 "__________________________"
 "_________________3________"
 "__________________________"
 "_________________2________"
 "__________________________"
 "_________________1________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

1-(5, -7)
2-(5, -5)
3-(5, -3)
4-(5, -1)
5-(3, -1)
6-(2, 0)
7-(1, 0)
8-(0, 0)
9-(0, 0)


26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_________________0________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________876__________"
 "________________5_4_______"
 "__________________________"
 "__________________3_______"
 "__________________________"
 "__________________2_______"
 "__________________________"
 "__________________1_______"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

1-(6, -7)
2-(6, -5)
3-(6, -3)
4-(6, -1)
5-(4, -1)
6-(2, -1)
7-(1, 0)
8-(0, 0)
9-(0, 0)


26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "________________0_________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________87___________"
 "_______________6_5_4______"
 "__________________________"
 "___________________3______"
 "__________________________"
 "___________________2______"
 "__________________________"
 "___________________1______"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

...

26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "____________0_____________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "_____________9_8_7_6_5_4__"
 "__________________________"
 "_______________________3__"
 "__________________________"
 "_______________________2__"
 "__________________________"
 "_______________________1__"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

1-(11, -7)
2-(11, -5)
3-(11, -3)
4-(11, -1)
5-(9, -1)
6-(7, -1)
7-(5, -1)
8-(3, -1)
9-(1, -1)


26-element Vector{String}:
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "___________0______________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "______________9_8_7_6_5_4_"
 "__________________________"
 "________________________3_"
 "__________________________"
 "________________________2_"
 "__________________________"
 "________________________1_"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"
 "__________________________"

1-(12, -7)
2-(12, -5)
3-(12, -3)
4-(12, -1)
5-(10, -1)
6-(8, -1)
7-(6, -1)
8-(4, -1)
9-(2, -1)


Set{Tuple{Int64, Int64}} with 32 elements:
  (23, -6)
  (23, -3)
  (4, -1)
  (13, -1)
  (15, -1)
  (2, -1)
  (10, -1)
  (18, -1)
  (21, -1)
  (23, -2)
  (5, -1)
  (16, -1)
  (20, -1)
  (12, -1)
  (8, -1)
  (17, -1)
  (1, -1)
  (19, -1)
  (0, -1)
  (22, -1)
  (6, -1)
  (23, -1)
  (-1, -1)
  (11, -1)
  (23, -5)
  ⋮ 

In [21]:
# task both
task2 = true
reachedPositions = Set{Tuple{Int64, Int64}}([(0,0)])
knotsCount = task2 ? 10 : 2
knotPositions = Vector{Tuple{Int64, Int64}}()
newKnotPositions = Vector{Tuple{Int64, Int64}}()
for _ in 1:knotsCount
  push!(knotPositions, (0,0))
  push!(newKnotPositions, (0,0))
end

for move in moves
  # displayKnots(knotPositions)
  (dx, dy) = normalize(move)
  for _ in 1:normInf(move)
    newKnotPositions[1] = knotPositions[1] .+ (dx, dy)
    
    for i in 1:knotsCount-1
      if normInf(knotPositions[i+1] .- newKnotPositions[i]) > 1
        newKnotPositions[i+1] = knotPositions[i+1] .+ normalizeInf(newKnotPositions[i] .- knotPositions[i+1])
      else newKnotPositions[i+1] = knotPositions[i+1]
      end
    end

    knotPositions = copy(newKnotPositions)
    push!(reachedPositions, knotPositions[knotsCount])
  end
end

reachedPositions |> length

6332